In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [48]:
import io 
from google.colab import files
car_data = files.upload()
car = pd.read_csv(io.BytesIO(car_data['CarPrice_Assignment.csv']))


Saving CarPrice_Assignment.csv to CarPrice_Assignment (4).csv


In [49]:
car.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [50]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [51]:
x = car.drop(["price","car_ID","CarName"],axis=1).copy()
y = car["price"].copy()

In [52]:
{column: list(x[column].unique()) for column in x.columns if x.dtypes[column]=='object'}

{'aspiration': ['std', 'turbo'],
 'carbody': ['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop'],
 'cylindernumber': ['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
 'doornumber': ['two', 'four'],
 'drivewheel': ['rwd', 'fwd', '4wd'],
 'enginelocation': ['front', 'rear'],
 'enginetype': ['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv'],
 'fuelsystem': ['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi'],
 'fueltype': ['gas', 'diesel']}

In [53]:
x["doornumber"]=x["doornumber"].apply(lambda x: 0 if x == 'two' else 1)

In [54]:
numeric_ordering ={'two':2,'three':3,'four':4,'five':5,'six':6,'eight':8,'twelve':12}

In [55]:
x["cylindernumber"] =x["cylindernumber"].replace(numeric_ordering)

In [56]:
def binary_encode(x, columns, positive_values):
    x =x.copy()
    for column, positive_value in zip(columns, positive_values):
        x[column] = x[column].apply(lambda x: 1 if x == positive_value else 0)
    return x

In [57]:
binary_features=['fueltype','aspiration','enginelocation',]
binary_positive_values=['diesel','turbo','front']
x=binary_encode(x,columns=binary_features,positive_values=binary_positive_values)

In [58]:
def onehot_encode(x, columns, prefixes):
    x = x.copy()
    for column, prefix in zip(columns, prefixes):
        dummies = pd.get_dummies(x[column], prefix=prefix)
        x=pd.concat([x, dummies], axis=1)
        x=x.drop(column, axis=1)
    return x

In [59]:
nominal_features=['carbody','drivewheel','enginetype','fuelsystem']
prefixes=['MK','BS','DW','ET','FS']
x=onehot_encode(x,columns=nominal_features,prefixes=prefixes)

In [60]:
x

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,cylindernumber,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,MK_convertible,MK_hardtop,MK_hatchback,MK_sedan,MK_wagon,BS_4wd,BS_fwd,BS_rwd,DW_dohc,DW_dohcv,DW_l,DW_ohc,DW_ohcf,DW_ohcv,DW_rotor,ET_1bbl,ET_2bbl,ET_4bbl,ET_idi,ET_mfi,ET_mpfi,ET_spdi,ET_spfi
0,3,0,0,0,1,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111,5000,21,27,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3,0,0,0,1,88.6,168.8,64.1,48.8,2548,4,130,3.47,2.68,9.0,111,5000,21,27,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,1,94.5,171.2,65.5,52.4,2823,6,152,2.68,3.47,9.0,154,5000,19,26,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,2,0,0,1,1,99.8,176.6,66.2,54.3,2337,4,109,3.19,3.40,10.0,102,5500,24,30,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,2,0,0,1,1,99.4,176.6,66.4,54.3,2824,5,136,3.19,3.40,8.0,115,5500,18,22,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,0,0,1,1,109.1,188.8,68.9,55.5,2952,4,141,3.78,3.15,9.5,114,5400,23,28,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
201,-1,0,1,1,1,109.1,188.8,68.8,55.5,3049,4,141,3.78,3.15,8.7,160,5300,19,25,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
202,-1,0,0,1,1,109.1,188.8,68.9,55.5,3012,6,173,3.58,2.87,8.8,134,5500,18,23,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
203,-1,1,1,1,1,109.1,188.8,68.9,55.5,3217,6,145,3.01,3.40,23.0,106,4800,26,27,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0


In [61]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,random_state=123)

In [62]:
scaler = StandardScaler()
scaler.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [63]:
x_train =pd.DataFrame(scaler.transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.transform(x_test),columns=x_test.columns)
part_x_train,x_val,part_y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=2)

In [64]:
model = Sequential()
model.add(Dense(12, input_dim=42, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='Adam', metrics=['mse','mae'])

In [65]:
history=model.fit(part_x_train,part_y_train,epochs=100,validation_data=(x_val,y_val),batch_size=512)

Epoch 1/100
1/1 [==============================] - 1s 864ms/step - loss: 263500912.0000 - mse: 263500912.0000 - mae: 14127.9404 - val_loss: 334184032.0000 - val_mse: 334184032.0000 - val_mae: 14966.4619
Epoch 2/100
1/1 [==============================] - 0s 68ms/step - loss: 263500272.0000 - mse: 263500272.0000 - mae: 14127.9229 - val_loss: 334183360.0000 - val_mse: 334183360.0000 - val_mae: 14966.4463
Epoch 3/100
1/1 [==============================] - 0s 84ms/step - loss: 263499664.0000 - mse: 263499664.0000 - mae: 14127.9072 - val_loss: 334182688.0000 - val_mse: 334182688.0000 - val_mae: 14966.4307
Epoch 4/100
1/1 [==============================] - 0s 83ms/step - loss: 263499024.0000 - mse: 263499024.0000 - mae: 14127.8877 - val_loss: 334181984.0000 - val_mse: 334181984.0000 - val_mae: 14966.4160
Epoch 5/100
1/1 [==============================] - 0s 66ms/step - loss: 263498352.0000 - mse: 263498352.0000 - mae: 14127.8691 - val_loss: 334181312.0000 - val_mse: 334181312.0000 - val_mae: 

In [66]:
model.evaluate(x_test,y_test)

2/2 [==============================] - 0s 8ms/step - loss: 151848208.0000 - mse: 151848208.0000 - mae: 10916.0293


[151848208.0, 151848208.0, 10916.029296875]

In [67]:

model.predict(x_test).sum()

287.18326